In [ ]:
print()

# **Summarize Private Documents Using RAG, LangChain, and LLMs**



---------

So, this project steps you through the fascinating world of LLMs and RAG, starting from the basics of what these technologies are, to building a practical application that can read and summarize documents for you. By the end of this tutorial, you have a working tool capable of processing the pile of documents on your desk, allowing you to focus on making meaningful contributions to your projects sooner.


## Background

### What is RAG?
One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as retrieval-augmented generation (RAG). RAG is a technique for augmenting LLM knowledge with additional data, which can be your own data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to public data up to the specific point in time that they were trained. If you want to build AI applications that can reason about private data or data introduced after a model’s cut-off date, you must augment the knowledge of the model with the specific information that it needs. The process of bringing and inserting the appropriate information into the model prompt is known as RAG.

LangChain has several components that are designed to help build Q&A applications and RAG applications, more generally.

### RAG architecture
A typical RAG application has two main components:

* **Indexing**: A pipeline for ingesting and indexing data from a source. This usually happens offline.

* **Retrieval and generation**: The actual RAG chain takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

The most common full sequence from raw data to answer looks like the following examples.


- **Indexing**
1. Load: First, you must load your data. This is done with [DocumentLoaders](https://python.langchain.com/docs/how_to/#document-loaders).

2. Split: [Text splitters](https://python.langchain.com/docs/how_to/#text-splitters) break large `Documents` into smaller chunks. This is useful both for indexing data and for passing it into a model because large chunks are harder to search and won’t fit in a model’s finite context window.

3. Store: You need somewhere to store and index your splits so that they can later be searched. This is often done using a [VectorStore](https://python.langchain.com/docs/how_to/#vector-stores) and [Embeddings](https://python.langchain.com/docs/how_to/embed_text/) model.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/WEE3pjeJvSZP0R7UL7CYTA.png" width="50%" alt="indexing"/> <br>
<span style="font-size: 10px;">[source](https://python.langchain.com/docs/tutorials/rag/)</span>


- **Retrieval and generation**
1. Retrieve: Given a user input, relevant splits are retrieved from storage using a retriever.
2. Generate: A ChatModel / LLM produces an answer using a prompt that includes the question and the retrieved data.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/SwPO26VeaC8VTZwtmWh5TQ.png" width="50%" alt="retrieval"/> <br>
<span style="font-size: 10px;">[source](https://python.langchain.com/docs/use_cases/question_answering/)</span>


## Objectives


 - Master the technique of splitting and embedding documents into formats that LLMs can efficiently read and interpret.
 - Know how to access and utilize different LLMs from IBM watsonx.ai, selecting the optimal model for their specific document processing needs.
 - Implement different retrieval chains from LangChain, tailoring the document retrieval process to support various purposes.
 - Develop an agent that uses integrated LLM, LangChain, and RAG technologies for interactive and efficient document retrieval and summarization, making conversations have memory.


In [ ]:
!pip install --user "chromadb"


In [ ]:
!pip install --user "wget == 3.2"

In [ ]:
!pip install --user "langchain==0.1.16"


In [ ]:
!pip install sentence-transformers

In [ ]:
pip install fastembed

In [ ]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import wget

## Preprocessing
### Load the document

The document, which is provided in a TXT format, outlines some company policies and serves as an example data set for the project.

This is the `load` step in `Indexing`.<br>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MPdUH7bXpHR5muZztZfOQg.png" width="50%" alt="split"/>


In [ ]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

In [ ]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

### Splitting the document into chunks

In this step, you are splitting the document into chunks, which is basically the `split` process in `Indexing`.
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/0JFmAV5e_mejAXvCilgHWg.png" width="50%" alt="split"/>


`LangChain` is used to split the document and create chunks. It helps you divide a long story (document) into smaller parts, which are called `chunks`, so that it's easier to handle.

For the splitting process, the goal is to ensure that each segment is as extensive as if you were to count to a certain number of characters and meet the split separator. This certain number is called `chunk size`. Let's set 1000 as the chunk size in this project. Though the chunk size is 1000, the splitting is happening randomly. This is an issue with LangChain. `CharacterTextSplitter` uses `\n\n` as the default split separator. You can change it by adding the `separator` parameter in the `CharacterTextSplitter` function; for example, `separator="\n"`.



In [ ]:
loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

From the ouput of print, you see that the document has been split into 16 chunks


### Embedding and storing
This step is the `embed` and `store` processes in `Indexing`. <br>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/u_oJz3v2cSR_lr0YvU6PaA.png" width="50%" alt="split"/>


In this step, you're taking the pieces of the story, your "chunks," converting the text into numbers, and making them easier for your computer to understand and remember by using a process called "embedding." Think of embedding like giving each chunk its own special code. This code helps the computer quickly find and recognize each chunk later on.

You do this embedding process during a phase called "Indexing." The reason why is to make sure that when you need to find specific information or details within your larger document, the computer can do so swiftly and accurately.


The following code creates a default embedding model from Hugging Face and ingests them to Chromadb.

When it's completed, print "document ingested".


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document



In [ ]:
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.embeddings import FastEmbedEmbeddings

FAISS, built by Meta, is a high-performance library that gives full control over indexing and similarity search. It's fast, scalable, and great for handling large datasets — but it’s low-level. There’s no built-in support for metadata, persistence, or filtering. If you want flexibility and performance, FAISS delivers — but you’ll be wiring things up yourself.

Chroma, on the other hand, is a higher-level vector database designed for developer experience. It handles embeddings, metadata, and storage with a simple API, and it integrates smoothly with LLM tooling like LangChain. It’s ideal for prototyping and getting retrieval working quickly. But the trade-off is less transparency and less control under the hood, which can become a limitation at scale.

In [ ]:

embeddings = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')


Up to this point, you've been performing the `Indexing` task. The next step is the `Retrieval` task.


## LLM model construction
Define parameters for the model.

The decoding method is set to `greedy` to get a deterministic output.

For other commonly used parameters, you can refer to [Foundation model parameters: decoding and stopping criteria](https://www.ibm.com/docs/en/watsonx-as-a-service?utm_source=skills_network&utm_content=in_lab_content_link&utm_id=Lab-RAG_v1_1711546843&topic=lab-model-parameters-prompting).


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, pipeline


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
#from langchain_huggingface import HuggingFacePipeline
from langchain.llms import HuggingFacePipeline


# ✅ 1. Load tokenizer and model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ 2. Define generation pipeline
hf_pipe = pipeline(
    "text2text-generation",  # For T5-style models
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.5
)

# ✅ 3. Wrap for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipe)


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_name = "deepset/roberta-base-squad2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


This completes the `LLM` part of the `Retrieval` task. <br>

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/UZXQ44Tgv4EQ2-mTcu5e-A.png" width="50%" alt="split"/>


## Integrating LangChain

LangChain has a number of components that are designed to help retrieve information from the document and build question-answering applications, which helps you complete the `retrieve` part of the `Retrieval` task. <br>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/M4WpkkMMbfK0Wkz0W60Jiw.png" width="50%" alt="split"/>


In the following steps, you create a simple Q&A application over the document source using LangChain's `RetrievalQA`.

Then, you ask the query "what is mobile policy?"


# LangChain RetrievalQA Breakdown

Below is a detailed, step-by-step explanation of what each line in your snippet does.

---

## 1. Import & Setup (Assumed)

```python
from langchain.chains import RetrievalQA
#### llm      – your Language Model instance (e.g. OpenAI, HuggingFace pipeline)
#### docsearch– your vector store or text index, already initialized


### Component-by-Component Breakdown

🔸 llm=llm
Refers to the language model being used.

This could be an OpenAI model, a Hugging Face model, etc.

The LLM is responsible for generating the final answer based on the context provided.

🔸 chain_type="stuff"

 - Specifies how the documents are passed to the LLM.
 - "stuff" means all the retrieved documents are stuffed into a single prompt along with the question.
 - Good for small document sets or when you’re under token limits.
 - Alternatives include map_reduce, refine, etc.

🔸 retriever=docsearch.as_retriever()
Converts your document index (e.g. FAISS, Chroma, etc.) into a retriever.

The retriever:

Converts the query into an embedding

Searches for the most relevant documents

Returns those documents to be used as context

🔸 return_source_documents=False
When True, it also returns the raw documents used to answer the question.

When False, it returns only the generated answer.

In this case, we only want the final answer string.

🔍 Retriever (Embedding Model)
Role: Find relevant documents based on the query.

How: Converts the query and documents into embeddings (vectors) and finds the most similar ones.

This uses: an embedding model (e.g., OpenAI’s text-embedding-ada, Hugging Face’s sentence-transformers, etc.).

🧠 LLM (Language Model)
Role: Acts as a decoder/generator — takes the retrieved documents and generates a coherent, natural language answer.

Does not: Do the retrieval or embedding matching.

Does: Read the docs passed to it and generate an answer based on them (this is where the chain_type="stuff" matters — all the docs are stuffed into the prompt).



In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

From the response, it seems fine. The model's response is the relevant information about the mobile policy from the document.

Now, try to ask a more high-level question.


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

## Dive deeper

This section dives deeper into how you can improve this application. You might want to ask "How to add the prompt in retrieval using LangChain?" <br>

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/bvw3pPRCYRUsv-Z2m33hmQ.png" width="50%" alt="split"/>


You use prompts to guide the responses from an LLM the way you want. For instance, if the LLM is uncertain about an answer, you instruct it to simply state, "I do not know," instead of attempting to generate a speculative response.

Let's see an example.


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can I eat in company vehicles?"
qa.invoke(query)

As you can see, the query is asking something that does not exist in the document. The LLM responds with information that actually is not true. You don't want this to happen, so you must add a prompt to the LLM.


### Using prompt template

In the following code, you create a prompt template using `PromptTemplate`.

`context` and `question` are keywords in the RetrievalQA, so LangChain can automatically recognize them as document content and query.


In [ ]:
prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

In [ ]:
prompt_template = """ If you dont know say "I don't know."

<context>
{context}
</context>

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

### Make the conversation have memory

Do you want your conversations with an LLM to be more like a dialogue with a friend who remembers what you talked about last time? An LLM that retains the memory of your previous exchanges builds a more coherent and contextually rich conversation.

Take a look at a situation in which an LLM does not have memory.

You start a new query, "What I cannot do in it?". You do not specify what "it" is. In this case, "it" means "company vehicles" if you refer to the last query.


In [ ]:
query = "What I cannot do in it?"
qa.invoke(query)

From the response, you see that the model does not have the memory because it does not provide the correct answer, which is something related to "smoking is not permitted in company vehicles."

To make the LLM have memory, you introduce the `ConversationBufferMemory` function from LangChain.


In [ ]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

Create a `ConversationalRetrievalChain` to retrieve information and talk with the LLM.


In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           chain_type="stuff",
                                           retriever=docsearch.as_retriever(),
                                           memory = memory,
                                           get_chat_history=lambda h : h,
                                           return_source_documents=False)

Create a `history` list to store the chat history.


In [ ]:
history = []

query = "What is mobile policy?"
result = qa.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

Append the previous query and answer to the history.


In [ ]:
history.append((query, result["answer"]))
query = "List points in it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

In [ ]:
history.append((query, result["answer"]))
query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

### Wrap up and make it an agent

The following code defines a function to make an agent, which can retrieve information from the document and has the conversation memory.


In [ ]:
def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                               chain_type="stuff",
                                               retriever=docsearch.as_retriever(),
                                               memory = memory,
                                               get_chat_history=lambda h : h,
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")

        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break

        result = qa({"question": query}, {"chat_history": history})

        history.append((query, result["answer"]))

        print("Answer: ", result["answer"])

In [ ]:
qa()